<a href="https://www.kaggle.com/code/ocanaydin/animal-classify-tfl?scriptVersionId=113934637" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
images_dir = "../input/animals10/raw-img"
import tensorflow as tf
import pathlib
!pip install split-folders

**SPLIT DATA AS TRAIN AND VALIDATION**

In [ ]:
import splitfolders
def split_data(data_path):
    data = pathlib.Path(data_path)
    splitfolders.ratio(data,output = "../outputs/Images/",seed = 42,ratio = (0.8,0.2),group_prefix = None)

In [ ]:
split_data(images_dir)

**PRE PROCESSING OF IMAGES**

In [ ]:
"""All images will be scaled with 1./255 to normalize between 0-1."""
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory("../outputs/Images/train",target_size = (150,150),
                                                   class_mode = "categorical",batch_size = 32,seed=42)
validation_generator = validation_datagen.flow_from_directory("../outputs/Images/val",target_size = (150,150),
                                                             class_mode = "categorical",batch_size = 32,
                                                              shuffle = False,seed = 42)

**CNN ARCHITECTURE**

**SEPERABLE 2D CONVOLUTION**

In [ ]:
model = tf.keras.models.Sequential()
"""(1)Convolution and batch normalization."""
model.add(tf.keras.layers.SeparableConv2D(32,(3,3),input_shape = (150,150,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
"""(2)Convolution and batch normalization."""
model.add(tf.keras.layers.SeparableConv2D(64,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
"""(3)Convolution and batch normalization."""
model.add(tf.keras.layers.SeparableConv2D(32,(3,3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
"""Fully connected layer and dropout."""
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256,activation = "relu"))
model.add(tf.keras.layers.Dropout(0.3))
"""Output layer"""
model.add(tf.keras.layers.Dense(10,activation = "relu"))

In [ ]:
model.summary()

**TRANSFER LEARNING ARCHITECTURE**

**(1)Use Xception as base model**

In [ ]:
from tensorflow.keras.applications.xception import Xception
base_model = Xception(include_top = False,weights = "imagenet",input_shape = (150,150,3),pooling = "max")
"""Freeze layers to stop updating the weights of Xception."""
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

**(2)Create an architecture to feed model.**

In [ ]:
"""Here we chose add_11 layer.We can start to update weights after add_11 layer in training."""
last_layer = base_model.get_layer("add_11")
print(last_layer.output_shape)

In [ ]:
classes = os.listdir(images_dir)
"""BatchNormalization and GlobalAveragePooling to reduce input dim to 1D."""
x = tf.keras.layers.BatchNormalization()(last_layer.output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
"""Fully connected layer."""
x = tf.keras.layers.Dense(128,activation = "relu")(x)
"""Dropout layer"""
x = tf.keras.layers.Dropout(0.3)(x)
"""Output layer"""
x = tf.keras.layers.Dense(len(classes),activation = "softmax")(x)
"""Finally,we can connect model end to end."""
model = tf.keras.models.Model(base_model.input,x)

In [ ]:
model.summary()

**COMPILE AND FIT MODEL**

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate = 0.001),loss = "categorical_crossentropy",metrics = ["acc"])
"""Callback class."""
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if epoch >= 10 and logs.get("acc") - logs.get("val_acc") >= 0.1:
            self.model.stop_training = True
            print("Model tends to be overfitting.Stop training.")
        elif logs.get("acc") > 0.95:
            self.model.stop_training = True
            print("Model tends to be overfitting.Stop training.")
callback = myCallback()
        

In [ ]:
history = model.fit(train_generator,epochs = 30,batch_size = 32,validation_data = validation_generator,
                   callbacks = [callback],verbose = 1)

**PLOT ACCURACY AND LOSS**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(11)
plt.plot(epochs,acc,label = "Training Accuracy")
plt.plot(epochs,val_acc,label = "Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(11)
plt.plot(epochs,loss,label = "Training Loss")
plt.plot(epochs,val_loss,label = "Validation Loss")
plt.legend()
plt.show()

**SAVE AND LOAD MODEL**

In [ ]:
model.save("animal_classify_TFL.h5")

In [ ]:
model1 = tf.keras.models.load_model("animal_classify_TFL.h5")

**GET THE IMAGES FROM INTERNET AND PROCESS THEM TO PREDICT**

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np

In [ ]:
def get_and_process(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img1 = img
    """Resize img to proper shape for model."""
    img = img.resize((150,150))
    """Convert img to numpy array,rescale it,expand dims and check vertically."""
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = x / 255
    x = np.expand_dims(x,axis = 0)
    img_tensor = np.vstack([x])
    return img1,img_tensor

**FINAL : PREDICT IMAGE**

In [ ]:
import matplotlib.pyplot as plt
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken",
             "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane",
             "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", 
             "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}

url = "https://i.natgeofe.com/k/ff49e0e1-20b6-4c4b-84c8-4ad196e312e4/eastern-gray-squirrel-closeup_square.jpg"
img1,test_img = get_and_process(url)
pred = model1.predict(test_img)
classes = list(train_generator.class_indices.keys())
print(f"Prediction is : {translate[classes[np.argmax(pred)]]}")
plt.imshow(img1)
plt.show()
print(pred)
print(classes)